In [4]:
import torch
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import modeling.backbone as backbone

In [5]:
# Load the saved model
net = backbone.net_factory(
    num_classes=None,  # No classifier needed
    flowpic_dim=32,
    with_dropout=True,
    projection_layer_dim=30,
)

net.features.fullyconnected[3] = torch.nn.Linear(120, 120)
net.classifier = torch.nn.Linear(120, 30)

artifacts_folder = Path("artifacts-mirage22-min10")
split_idx = 0
weights_path = artifacts_folder / f"best_model_weights_pretrain_split_{split_idx}.pt"
net.load_weights(weights_path)
net.eval()  # Set to evaluation mode

print("✅ Model successfully loaded.")

✅ Model successfully loaded.


In [6]:
import modeling.dataprep as dataprep
from tcbench import DATASETS, MODELING_DATASET_TYPE

# ✅ Define augmentation configuration (from pretrain())
aug_config = dict(changertt={}, timeshift={})

# ✅ Reload dataset with required augmentation configuration
dset_train, dset_val = dataprep.load_dataset(
    dataset_name=DATASETS.MIRAGE19,
    dataset_type=MODELING_DATASET_TYPE.TRAIN_VAL,
    split_idx=0,
    flowpic_dim=32,
    flowpic_block_duration=15,
    aug_config=aug_config,  # ✅ Pass required augmentation config
    aug_samples=2,
    aug_when_loading=False,
    n_workers=50,
    logger=None,
    dataset_minpkts=10,
    max_samples_per_class=-1,
    seed=12345,
    train_val_split_ratio=0.8,
)

# ✅ Create DataLoader
data_loader = torch.utils.data.DataLoader(
    dset_train,
    batch_size=32,
    shuffle=True,
)

print("✅ Dataset loaded successfully!")

/home/ev357/miniconda3/envs/tcbench/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

loaded: /home/ev357/miniconda3/envs/tcbench/lib/python3.10/site-packages/tcbench/libtcdatasets/datasets/mirage19/preprocessed/imc23/mirage19_filtered_minpkts10.parquet


loaded: /home/ev357/miniconda3/envs/tcbench/lib/python3.10/site-packages/tcbench/libtcdatasets/datasets/mirage19/preprocessed/imc23/mirage19_filtered_minpkts10_splits.parquet
dataset samples count
                             train  val
app                                    
de.motain.iliga               6079  675
com.waze                      5844  649
com.duolingo                  3712  413
it.subito                     3482  387
com.contextlogic.wish         3181  353
com.accuweather.android       3027  336
com.joelapenna.foursquared    2938  326
com.spotify.music             2673  297
com.dropbox.android           2583  287
com.facebook.katana           2331  259
com.iconology.comics          2278  253
com.twitter.android           2272  252
com.google.android.youtube    2209  246
com.pinterest                 1984  221
com.tripadvisor.tripadvisor   1662  185
com.facebook.orca             1444  161
com.viber.voip                1310  146
com.trello                    1197  133
com

In [15]:
# Function to extract embeddings
def extract_embeddings(model, dataloader, device="cuda:0"):
    model.to(device)
    embeddings = []
    labels = []
    
    with torch.no_grad():  # No gradients needed
        for batch in dataloader:
            inputs = torch.stack(batch[0]).squeeze(2).to(device).float()  # Remove extra dim
            targets = batch[1]  # Labels are already a tensor
            
            features = model.features(inputs)  # Extract embeddings
            embeddings.append(features.cpu())  # Move to CPU for storage
            labels.append(targets.cpu())  # Ensure labels are also on CPU
    
    return torch.cat(embeddings), torch.cat(labels)

# Extract embeddings
embeddings, labels = extract_embeddings(net, data_loader, device="cpu")
print("Embeddings shape:", embeddings.shape)  # (num_samples, embedding_dim)
print("Labels shape:", labels.shape)


RuntimeError: Given groups=1, weight of size [6, 1, 5, 5], expected input[2, 32, 32, 32] to have 1 channels, but got 32 channels instead

In [12]:
for batch in data_loader:
    print(type(batch), len(batch))  # Print type and length
    print(type(batch[0]), type(batch[1]))  # Print type of each element
    break  # Only check the first batch


<class 'list'> 2
<class 'list'> <class 'torch.Tensor'>
